In [10]:
import os
from dotenv import load_dotenv
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
load_dotenv()

# SCOPES: Gmail read-only
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
token_path = os.getenv("token_path")
credentials_path = os.getenv("credentials_path")

In [70]:
def get_message_metadata(service, user_id='me', max_results=5):
    results = service.users().messages().list(userId=user_id, maxResults=max_results, q="after:2025/08/08").execute()
    messages = results.get('messages', [])

    for _, msg in enumerate(messages):
        msg_data = service.users().messages().get(userId=user_id, id=msg['id'], format='metadata', metadataHeaders=['Subject', 'From', 'Date']).execute()
        headers = msg_data.get('payload', {}).get('headers', [])

        email_info = {header['name']: header['value'] for header in headers if header['name'] in ['Subject', 'From', 'Date']}
        print(f"From: {email_info.get('From')}")
        print(f"Subject: {email_info.get('Subject')}")
        print(f"Date: {email_info.get('Date')}")

In [17]:
import time, gzip, msgspec
import pandas as pd
path = "C:/airflow-docker/data/15-08-2025-10-41.json.gz"
#Decompress and load
with gzip.open(path, 'rb') as f:
    decompressed_bytes = f.read()
decompressed_data = msgspec.msgpack.decode(decompressed_bytes)

df =pd.DataFrame(decompressed_data)
df.head(2)

,Id,Payload
0,1988691375a9beb7,"{'partId': '', 'mimeType': 'multipart/alternat..."
1,19885fd9b18c5065,"{'partId': '', 'mimeType': 'multipart/mixed', ..."


In [20]:
from dags.utils import extract_headers, decode_body 
for payload in decompressed_data["Payload"][:10]:
    subject, sender = extract_headers(payload)
    body_text = decode_body(payload, prefer_plain=True).strip()

    print(f"\n--- Email ---")
    print(f"From: {sender}")
    print(f"Subject: {subject}")
    #print(f"Body:\n{body_text}")


--- Email ---
From: Braintrust <community@usebraintrust.com>
Subject: Enhance Your Career with Braintrust's Career Help

--- Email ---
From: LeetCode <no-reply@leetcode.com>
Subject: LeetCode Weekly Digest

--- Email ---
From: Kaggle <no-reply@kaggle.com>
Subject: Competition Launch: Open Model Red-Teaming Challenge

--- Email ---
From: Naukri <info@naukri.com>
Subject: Status of your job application has changed

--- Email ---
From: LinkedIn <linkedin@e.linkedin.com>
Subject: Methari, enjoy 50% off LinkedIn Premium for 2 months

--- Email ---
From: Glassdoor Jobs <noreply@glassdoor.com>
Subject: Data Engineer at Blackstraw and 12 more jobs in Chennai for you. Apply Now.

--- Email ---
From: Glassdoor Jobs <noreply@glassdoor.com>
Subject: Ai/ml Excutive at Wellorgs Infotech Pvt. Ltd. and 12 more jobs in Remote, India for you. Apply Now.

--- Email ---
From: LinkedIn <jobs-noreply@linkedin.com>
Subject: New jobs similar to Data Scientist at Tallento.ai

--- Email ---
From: Glassdoor Job